In [1]:
%pip install -qU \
    replicate \
    langchain \
    sentence_transformers \
    pdf2image \
    pdfminer \
    pdfminer.six \
    unstructured


Note: you may need to restart the kernel to use updated packages.


In [2]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

In [3]:
from langchain.llms import CTransformers

llama_model = CTransformers(
    model = "C:\\Users\\adity\\OneDrive\\Documents\\llama-2-7b-chat.ggmlv3.q2_K.bin",
    model_type = "llama",
    config = {'max_new_tokens':1000,
              'temperature':0.75,
              'context_length':2000}
)

In [4]:
#load the external data source
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader('C:\\Users\\adity\\Desktop\\Chat UI\\Data')
documents = loader.load()

#Get text splits from document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

#Use the embedding model
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2" # embedding model
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

#Use vector store to store embeddings
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [5]:
def md(t):
  display(Markdown(t))

In [6]:
# Query 1
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True)

chat_history = []
query = "a diet for a healthy lifestyle"
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])

c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 The Mediterranean diet is a great choice for anyone looking to maintain a healthy lifestyle. It focuses on eating whole, fresh foods and incorporates small amounts of meat for protein. This meal plan offers a balanced approach to eating and can be followed for the long term.
Unhelpful Answer: I don't know.

In [10]:
# Query 2
query = "hat is your name?"
result1 = chain({"question": query, "chat_history": chat_history})
md(result1['answer'])

 I don't know, I don't have access to personal information about individuals unless it is explicitly stated in the text.

In [11]:
#General Query
query = input()
result2 = chain({"question": query, "chat_history": chat_history})
md(result2['answer'])

 
The answer is 3.

Instructions:

1. Mix all ingredients together except for the ginger, lemоn juice and cayenne pepper in a blender or food processor until well combined.
2. Add the grated ginger, crushed cloves, xylitol, green lime juice, green lime zest and cayenne pepper to the mixture and blend until well combined.
3. Pour into small cups or bowls for dipping. Serve with sushi dip, if desired.

In [13]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Function to get BERT embeddings
def get_embedding(text, model_name='bert-base-uncased'):
    # Initialize tokenizer and model for BERT
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Prepare the text for BERT using the tokenizer
    # This turns the text into a format BERT can understand
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Disable gradient calculation for performance
    with torch.no_grad():
        # Get the model's output, which includes embeddings
        outputs = model(**inputs)

    # The embeddings are averaged to get a single vector per input
    embeddings = outputs.last_hidden_state.mean(1)
    return embeddings.numpy()  # Convert tensor to NumPy array for compatibility with cosine_similarity

In [14]:
def compare_responses(response1, response2, model_name='bert-base-uncased'):
    # Convert responses to embeddings
    emb1 = get_embedding(response1, model_name)
    emb2 = get_embedding(response2, model_name)

    # Calculate the cosine similarity between the two sets of embeddings
    similarity = cosine_similarity(emb1, emb2)
    return similarity[0][0]


# Compare the two responses and print the similarity score
similarity_score = compare_responses(result1['answer'], result2['answer'])
print("Cosine similarity score:", similarity_score)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adity\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Cosine similarity score: 0.5400625


In [15]:
pip install gradio

  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
Using cached python_multipart-0.0.9-py3-none-any.whl (22 kB)
Using cached typer-0.12.3-py3-none-any.whl (47 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.7
    Uninstalling python-multipart-0.0.7:
      Successfully uninstalled python-multipart-0.0.7
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chainlit 1.0.506 requires dataclasses_json<0.6.0,>=0.5.7, but you have dataclasses-json 0.6.4 which is incompatible.
chainlit 1.0.506 requires fastapi<0.111.0,>=0.110.1, but you have fastapi 0.109.2 which is incompatible.
chainlit 1.0.506 requires python-dotenv<2.0.0,>=1.0.0, but you have python-dotenv 0.21.1 which is incompatible.
chainlit 1.0.506 requires starlette<0.38.0,>=0.37.2, but you have starlette 0.36.3 which is incompatible.
chainlit 1.0.506 requires uvicorn<0.26.0,>=0.25.0, but you have uvicorn 0.27.1 which is incompatible.


In [16]:
import gradio as gr
from langchain.llms import CTransformers, Replicate
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os

In [17]:
REPLICATE_API_TOKEN = "r8_TpNRKTh8GINVY78f3owhndEgoOp9WmB2CuHFX"
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [18]:
model_names = {
    "7B Model (LLaMa 2)": CTransformers,
}


In [19]:
loader = PyPDFDirectoryLoader('C:\\Users\\adity\\Desktop\\Chat UI\\Data')
documents = loader.load()

# Process the loaded documents into manageable text chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# Setup the embedding model with Sentence Transformers and initialize vector storage with FAISS
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
vectorstore = FAISS.from_documents(all_splits, embeddings)

# Define the response function for the chatbot, which uses selected model and configuration
def chatbot_response(model_type, user_input, max_new_tokens, temperature, top_p, context_length):
    # Select the model based on user input and initialize with specified parameters
    if model_type == "7B Model (LLaMa 2)":
        model = CTransformers(
            model="./llama-2-7b-chat.ggmlv3.q8_0.bin",
            model_type="llama",
            config={'max_new_tokens': int(max_new_tokens), 'temperature': float(temperature), 'context_length': int(context_length)}
        )

    # Create a conversational retrieval chain with the selected model and query it
    chain = ConversationalRetrievalChain.from_llm(model, vectorstore.as_retriever(), return_source_documents=True)
    chat_history = []
    result = chain({"question": user_input, "chat_history": chat_history})
    return result['answer']

# Setup the Gradio interface with dynamic inputs based on model type
interface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        gr.Radio(list(model_names.keys()), label="Model Type"),
        gr.Textbox(lines=2, placeholder="Type your question here..."),
        gr.Number(label="Max New Tokens", value=1000, step=1),
        gr.Number(label="Temperature", value=0.75, step=0.01),
    ],
    outputs="text",
    title="DietBot",
)

# Run the interface, making it available as a web application
interface.launch()

c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\gradio\utils.py:924: UserWarning: Expected 6 arguments for function <function chatbot_response at 0x000001CC0606DEE0>, received 4.
  warnings.warn(
c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\gradio\utils.py:928: UserWarning: Expected at least 6 arguments for function <function chatbot_response at 0x000001CC0606DEE0>, received 4.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.27.0, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.


c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\gradio\helpers.py:941: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Traceback (most recent call last):
  File "c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\gradio\blocks.py", line 1788, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\gradio\blocks.py", line 1340, in call_function
    prediction = await anyio.to_thread.run_sync(
           